In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/딥러닝')

In [3]:
import pandas as pd
data_hour = pd.read_csv("dat2.csv", encoding = 'cp949')

In [4]:
data_hour

,code,x,y,date,tmp,uwind,vwind,prec
0,309,126.6866,37.9464,2021-01-01 01:00:00,-15.5,-0.553643,-0.950515,0.0
1,309,126.6866,37.9464,2021-01-01 02:00:00,-15.7,-0.492007,0.983834,0.0
2,309,126.6866,37.9464,2021-01-01 03:00:00,-16.4,-0.425665,-0.677355,0.0
3,309,126.6866,37.9464,2021-01-01 04:00:00,-16.9,0.300000,0.000000,0.0
4,309,126.6866,37.9464,2021-01-01 05:00:00,-16.8,0.400000,0.000000,0.0
...,...,...,...,...,...,...,...,...
590984,967,126.6128,37.1189,2021-12-18 19:00:00,3.0,-8.724843,-8.092411,0.0
590985,967,126.6128,37.1189,2021-12-18 20:00:00,4.1,8.441201,-10.148208,0.0
590986,967,126.6128,37.1189,2021-12-18 21:00:00,4.3,6.145206,10.073552,0.0
590987,967,126.6128,37.1189,2021-12-18 22:00:00,4.6,-13.819786,-5.010341,0.0


In [5]:
data_hour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590989 entries, 0 to 590988
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   code    590989 non-null  int64  
 1   x       590989 non-null  float64
 2   y       590989 non-null  float64
 3   date    590989 non-null  object 
 4   tmp     590989 non-null  float64
 5   uwind   590989 non-null  float64
 6   vwind   590989 non-null  float64
 7   prec    590989 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 36.1+ MB


In [ ]:
# test = data_hour.groupby('code').tail(24)
# train = data_hour.drop(test.index)

> 완성할 데이터셋

In [6]:
true_df = pd.DataFrame(columns = ['code', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10',
                             't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20',
                             't21', 't22', 't23', 't24'])
pred_df = pd.DataFrame(columns = ['code', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10',
                             't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20',
                             't21', 't22', 't23', 't24'])

> 변수 정규화

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.python.keras.callbacks import EarlyStopping

def generator(dataset, target, start_index, end_index, 
              history_size, target_size, step, single_step = False):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size
  
  for i in range(start_index, end_index):
    indices = range(i - history_size, i, step)
    data.append(dataset.iloc[indices,])

    if single_step:
      labels.append(target[i + target_size])
    else:
      labels.append(target[i : i + target_size])
  
  return np.array(data), np.array(labels)

In [8]:
def plot_train_history(history, title):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs = range(len(loss))
    
    plt.figure()
    
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title(title)
    plt.legend()
    
    plt.show()

In [9]:
def create_time_steps(length):
  return list(range(-length, 0))

In [10]:
import matplotlib.pyplot as plt

def show_plot(plot_data, delta, title):
    labels = ['History', 'True Future', 'Model Prediction']
    marker = ['.-', 'rx', 'go']
    time_steps = create_time_steps(plot_data[0].shape[0])
    if delta:
        future = delta
    else:
        future = 0
        
    plt.title(title)
    for i, x in enumerate(plot_data):
        if i:
            plt.plot(future, plot_data[i], marker[i], markersize=10, label=labels[i])
        else:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
    
    plt.legend()
    plt.xlim([time_steps[0], (future+5)*2])
    plt.xlabel('Time-Step')
    return plt

In [11]:
def multi_step_plot(history, true_future, prediction):
    plt.figure(figsize=(12,6))
    num_in = create_time_steps(len(history))
    num_out = len(true_future)
    
    plt.plot(num_in, np.array(history[:,0]), label='History')
    plt.plot(np.arange(num_out)/STEP, np.array(true_future),'bo', label='True Future')
    
    if prediction.any():
        plt.plot(np.arange(num_out)/STEP, np.array(prediction), 'ro', label='Predicted Future')
    plt.legend(loc='upper left')
    plt.show()

In [ ]:
past_history = 24 * 5 # 24 * 5
future_target = 24 # 12
STEP = 1

BATCH_SIZE = 256
BUFFER_SIZE = 100

EVALUATION_INTERVAL = 200
EPOCHS = 10

tf.random.set_seed(42)

for i in data_hour.code.unique():
  
  dat = data_hour[data_hour.code == i]
  dat = dat.set_index('date')
  dat.index.names = [None]
  dat.drop(['code', 'x', 'y'], axis = 1, inplace = True)

  # 데이터 정규화
  hour_mean = dat.mean()
  hour_std = dat.std()
  dat = (dat - hour_mean) / hour_std

  # 데이터 처리
  n = dat.shape[0]
  x_train_multi, y_train_multi = generator(dat, dat.tmp, 0, 6000,
                                           past_history, future_target, STEP)
  x_val_multi, y_val_multi = generator(dat, dat.tmp, 6000, n-145,
                                       past_history, future_target, STEP)
  x_test_multi, y_test_multi = generator(dat, dat.tmp, n-145, None,
                                         past_history, future_target, STEP)
  
  train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
  train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

  val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))
  val_data_multi = val_data_multi.batch(BATCH_SIZE).repeat()

  test_data_multi = tf.data.Dataset.from_tensor_slices((x_test_multi, y_test_multi))
  test_data_multi = test_data_multi.batch(BATCH_SIZE).repeat()

  # RNN
  multi_step_model = tf.keras.models.Sequential()
  multi_step_model.add(tf.keras.layers.LSTM(8, input_shape = x_train_multi.shape[-2:],
                                          return_sequences = True))
  multi_step_model.add(tf.keras.layers.LSTM(4, activation = 'relu'))
  multi_step_model.add(tf.keras.layers.Dense(24))
  multi_step_model.compile(optimizer = tf.keras.optimizers.RMSprop(clipvalue = 1.0), loss = 'mae')

  multi_step_history = multi_step_model.fit(train_data_multi, epochs = EPOCHS,
                                            steps_per_epoch = EVALUATION_INTERVAL,
                                            validation_data = val_data_multi,
                                            validation_steps = 50,
                                            callbacks = [EarlyStopping(monitor = 'val_loss', patience = 5)])
  
  for x,y in test_data_multi.take(1):
    pred = multi_step_model.predict(x)[0]
    temp_df = pd.DataFrame([np.append(i, pred).tolist()],
                     columns = pred_df.columns)
    pred_df = pred_df.append(temp_df)
    
    true = y[0]
    temp_df = pd.DataFrame([np.append(i, true).tolist()],
                         columns = true_df.columns)
    true_df = true_df.append(temp_df)

Epoch 1/10
200/200 [==============================] - 23s 98ms/step - loss: 0.8639 - val_loss: 0.6280
Epoch 2/10
200/200 [==============================] - 19s 93ms/step - loss: 1.2297 - val_loss: 0.6114
Epoch 3/10
200/200 [==============================] - 19s 93ms/step - loss: 2785.4565 - val_loss: 0.5937
Epoch 4/10
200/200 [==============================] - 19s 96ms/step - loss: 12848.7793 - val_loss: 0.5838
Epoch 5/10
200/200 [==============================] - 20s 101ms/step - loss: 35329.6055 - val_loss: 0.5624
Epoch 6/10
200/200 [==============================] - 19s 94ms/step - loss: 461422.8750 - val_loss: 0.5315
Epoch 7/10
200/200 [==============================] - 18s 90ms/step - loss: 2241946.0000 - val_loss: 1282.9521
Epoch 8/10
200/200 [==============================] - 19s 93ms/step - loss: 106048.7109 - val_loss: 0.5081
Epoch 9/10
200/200 [==============================] - 18s 91ms/step - loss: 2634.5715 - val_loss: 0.4887
Epoch 10/10
1/1 [==============================]

1/1 [==============================] - 1s 829ms/step
Epoch 1/10
200/200 [==============================] - 22s 95ms/step - loss: 0.7869 - val_loss: 0.4778
Epoch 2/10
200/200 [==============================] - 20s 98ms/step - loss: 0.5908 - val_loss: 0.3735
Epoch 3/10
200/200 [==============================] - 19s 94ms/step - loss: 0.4205 - val_loss: 0.3052
Epoch 4/10
200/200 [==============================] - 19s 94ms/step - loss: 0.3321 - val_loss: 0.2836
Epoch 5/10
200/200 [==============================] - 19s 94ms/step - loss: 0.3021 - val_loss: 0.2672
Epoch 6/10
200/200 [==============================] - 19s 94ms/step - loss: 0.2880 - val_loss: 0.2631
Epoch 7/10
200/200 [==============================] - 19s 94ms/step - loss: 0.2812 - val_loss: 0.2677
Epoch 8/10
200/200 [==============================] - 21s 103ms/step - loss: 0.2792 - val_loss: 0.2602
Epoch 9/10
200/200 [==============================] - 18s 92ms/step - loss: 0.2740 - val_loss: 0.2554
Epoch 10/10
200/200 [=======

1/1 [==============================] - 1s 503ms/step
Epoch 1/10
200/200 [==============================] - 24s 107ms/step - loss: 0.8003 - val_loss: 0.5276
Epoch 2/10
200/200 [==============================] - 19s 96ms/step - loss: 0.6003 - val_loss: 0.4107
Epoch 3/10
200/200 [==============================] - 19s 95ms/step - loss: 0.4446 - val_loss: 0.3516
Epoch 4/10
200/200 [==============================] - 19s 93ms/step - loss: 0.3560 - val_loss: 0.3570
Epoch 5/10
200/200 [==============================] - 19s 96ms/step - loss: 0.3318 - val_loss: 0.3359
Epoch 6/10
200/200 [==============================] - 19s 94ms/step - loss: 0.3195 - val_loss: 0.3286
Epoch 7/10
200/200 [==============================] - 19s 96ms/step - loss: 0.3156 - val_loss: 0.3440
Epoch 8/10
200/200 [==============================] - 20s 100ms/step - loss: 0.3139 - val_loss: 0.3253
Epoch 9/10
200/200 [==============================] - 19s 95ms/step - loss: 0.3046 - val_loss: 0.3174
Epoch 10/10
1/1 [==========

In [ ]:
true_df.to_csv("true_df.csv",index = False)
pred_df.to_csv("pred_df.csv",index = False)

In [ ]:
past_history = 24 * 5 # 24 * 5
future_target = 24 # 12
STEP = 1

BATCH_SIZE = 256 # 256
BUFFER_SIZE = 100

EVALUATION_INTERVAL = 200
EPOCHS = 30

tf.random.set_seed(42)

dat = data_hour[data_hour.code == 309]
dat = dat.set_index('date')
dat.index.names = [None]
dat.drop(['code', 'x', 'y'], axis = 1, inplace = True)

# 데이터 정규화
hour_mean = dat.mean()
hour_std = dat.std()
dat = (dat - hour_mean) / hour_std

# 데이터 처리
n = dat.shape[0]
x_train_multi, y_train_multi = generator(dat, dat.tmp, 0, 6000,
                                           past_history, future_target, STEP)
x_val_multi, y_val_multi = generator(dat, dat.tmp, 6000, n-145,
                                           past_history, future_target, STEP)
x_test_multi, y_test_multi = generator(dat, dat.tmp, n-145, None,
                                           past_history, future_target, STEP)

train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))
val_data_multi = val_data_multi.batch(BATCH_SIZE).repeat()

test_data_multi = tf.data.Dataset.from_tensor_slices((x_test_multi, y_test_multi))
test_data_multi = test_data_multi.batch(BATCH_SIZE).repeat()

# RNN
multi_step_model = tf.keras.models.Sequential()
multi_step_model.add(tf.keras.layers.LSTM(8, input_shape = x_train_multi.shape[-2:],
                                          return_sequences = True))
multi_step_model.add(tf.keras.layers.LSTM(4, activation = 'relu'))
multi_step_model.add(tf.keras.layers.Dense(24))
multi_step_model.compile(optimizer = tf.keras.optimizers.RMSprop(clipvalue = 1.0), loss = 'mae')

multi_step_history = multi_step_model.fit(train_data_multi, epochs = EPOCHS,
                                          steps_per_epoch = EVALUATION_INTERVAL,
                                            validation_data = val_data_multi,
                                            validation_steps = 50,
                                            callbacks = [EarlyStopping(monitor = 'val_loss', patience = 6)])

Epoch 1/30
200/200 [==============================] - 27s 116ms/step - loss: 0.8639 - val_loss: 0.6280
Epoch 2/30
200/200 [==============================] - 21s 104ms/step - loss: 1.2297 - val_loss: 0.6114
Epoch 3/30
200/200 [==============================] - 22s 109ms/step - loss: 2785.4565 - val_loss: 0.5937
Epoch 4/30
200/200 [==============================] - 32s 163ms/step - loss: 12848.7793 - val_loss: 0.5838
Epoch 5/30
200/200 [==============================] - 20s 102ms/step - loss: 35329.6055 - val_loss: 0.5624
Epoch 6/30
200/200 [==============================] - 21s 103ms/step - loss: 461422.8750 - val_loss: 0.5315
Epoch 7/30
200/200 [==============================] - 21s 106ms/step - loss: 2241946.0000 - val_loss: 1282.9521
Epoch 8/30
200/200 [==============================] - 22s 110ms/step - loss: 106048.7109 - val_loss: 0.5081
Epoch 9/30
200/200 [==============================] - 21s 104ms/step - loss: 2634.5715 - val_loss: 0.4887
Epoch 10/30
200/200 [===================